In [7]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, BigInteger, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import os
from libv3.utils import *
import pandas as pd
import json
import shutil

# Define the base for the declarative model
Base = declarative_base()

# Define the Event class which will be mapped to the events table in the database
class Event(Base):
    __tablename__ = 'events'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    
    name: Mapped[str] = mapped_column(String(50), nullable=False)
    timestamp: Mapped[int] = mapped_column(BigInteger, nullable=False)
    file_number: Mapped[int] = mapped_column(ForeignKey("file_config.id")) 

    config: Mapped["File_config"] = relationship(back_populates="events")
    


class File_config(Base):
    __tablename__ = 'file_config'

    id: Mapped[int] = mapped_column(primary_key=True)
    code_base: Mapped[str] = mapped_column(String(50), nullable=False)
    version: Mapped[int] = mapped_column(Integer, nullable=False)
    behaviour: Mapped[str] = mapped_column(String(50), nullable=False)
    trial_num: Mapped[int] = mapped_column(Integer, nullable=False)

    events: Mapped[List["Event"]] = relationship(back_populates="config")

# # Function to add events to the database
# def add_events(event_list):
#     for event_name, event_timestamp in event_list:
#         event = Event(name=event_name, timestamp=event_timestamp)
#         session.add(event)
#     session.commit()





/var/folders/nw/443sknns78l256x46fjj6r0c0000gn/T/ipykernel_31192/1128378996.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('wolfnet_config')   ### config stored in libv3/exp_config.txt
TRIAL = 1

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trial{TRIAL}'

print(log_path)

#### file to display
trace_file = 14

print('file number:', trace_file)

CODE: wolfnet
BEHAVIOUR: normal
THREAD: single
VER: 3
../trace_data/wolfnet/single_thread/version_3/normal/trial1
file number: 14


In [9]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

[]
['../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace10', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace11', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace12', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace13', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace14', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace15', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace16', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace17', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace18', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace19', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace20', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace21', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace22', '../trace_data/wolfnet/single_thread/version_3/normal/trial1/trace23', '.

### Combine files from trials

In [10]:
##### combine files
paths_traces.sort(key=lambda x: int(x.split('/')[-1][5:]))
last_file_num = int(paths_traces[-1].split('/')[-1][5:])
all_content = []
for trace_path in paths_traces:
    content = read_traces(trace_path)   
    all_content.extend(content)
    # print(content)

# path_to_save = os.path.dirname(os.path.dirname(trace_path))
# with open(path_to_save+f'/trace_trial{TRIAL}', 'w') as f:
#     json.dump(all_content, f)
#     # print('Saved trace:', path_to_save+f'/trace_trial{TRIAL})

# varlist = read_json(os.path.join(os.path.dirname(trace_path),f'varlist{last_file_num}'))
# # print(varlist)
# with open(path_to_save+f'/varlist_trial{TRIAL}.json', 'w') as f:
#     json.dump(varlist, f)



'../trace_data/wolfnet/single_thread/version_3/normal'

In [25]:
path_to_content = os.path.dirname(log_path)+f'/trace_trial{TRIAL}'
content_towrite = read_traces(path_to_content)
# print(content_towrite)
# Create an SQLite database (or connect to it if it already exists)
database_url = 'sqlite:///events.db'
engine = create_engine(database_url, echo=True)

# Create all tables in the database
Base.metadata.create_all(engine)

# # Create a configured "Session" class
Session = sessionmaker(bind=engine)

new_log_instance = File_config(code_base=CODE, version=VER, behaviour=BEHAVIOUR, trial_num=TRIAL)
print(new_log_instance.id)
# # #### since we use session.begin(), it will commit and close the session automatically as the with block ends
with Session.begin() as session:
    session.add(new_log_instance)
    session.commit()

with Session.begin() as session:
    for event_name, event_timestamp in all_content:
        event = Event(name=event_name, timestamp=event_timestamp, config=new_log_instance)
        session.add(event)
        # break



2024-07-08 13:57:51,689 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 13:57:51,690 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2024-07-08 13:57:51,690 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-08 13:57:51,690 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("file_config")
2024-07-08 13:57:51,691 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-08 13:57:51,691 INFO sqlalchemy.engine.Engine COMMIT
None
2024-07-08 13:57:51,692 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 13:57:51,696 INFO sqlalchemy.engine.Engine INSERT INTO file_config (code_base, version, behaviour, trial_num) VALUES (?, ?, ?, ?)
2024-07-08 13:57:51,698 INFO sqlalchemy.engine.Engine [generated in 0.00246s] ('wolfnet', 3, 'normal', 1)
2024-07-08 13:57:51,701 INFO sqlalchemy.engine.Engine COMMIT
2024-07-08 13:57:53,427 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 13:57:53,429 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_

In [23]:
#### query the database

with Session() as session:
    results = session.query(File_config).where(File_config.id == 1).all()
    print(results[0].id)
    print(results[0].code_base)
    print(results[0].version)
    print(results[0].behaviour)
    print(results[0].trial_num)
    print(results[0].events)

2024-07-08 13:54:08,699 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-08 13:54:08,700 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_config.code_base AS file_config_code_base, file_config.version AS file_config_version, file_config.behaviour AS file_config_behaviour, file_config.trial_num AS file_config_trial_num 
FROM file_config 
WHERE file_config.id = ?
2024-07-08 13:54:08,700 INFO sqlalchemy.engine.Engine [cached since 115.4s ago] (1,)
1
wolfnet
3
normal
1
2024-07-08 13:54:08,703 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.name AS events_name, events.timestamp AS events_timestamp, events.file_number AS events_file_number 
FROM events 
WHERE ? = events.file_number
2024-07-08 13:54:08,704 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1,)
[]
2024-07-08 13:54:08,705 INFO sqlalchemy.engine.Engine ROLLBACK
